In [379]:
import sys, os
from bokeh.palettes import brewer
import numpy as np
import pandas as pd
from IPython.display import HTML
from copy import copy
import pandas as pd 
sys.path.append(".")
from bokeh.models.filters import GroupFilter
from numpy import source
import lib.datasources
from datetime import date, datetime, timedelta
from bokeh.tile_providers import get_provider, WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM

from bokeh.models.tickers import DaysTicker
from bokeh.transform import cumsum, factor_cmap
from bokeh.palettes import *
#from bokeh.palettes import Spectral3, Spectral5, Viridis256, Category20c
from bokeh.layouts import widgetbox, layout, column, row
from bokeh.models import LinearColorMapper, LogColorMapper
from bokeh.models import ColumnDataSource, GeoJSONDataSource, TableColumn, DataTable, BooleanFilter, CDSView
from bokeh.models import CustomJS, Spinner
from bokeh.models import DateFormatter, HTMLTemplateFormatter, DatetimeTickFormatter, NumeralTickFormatter, MonthsTicker, YearsTicker
from bokeh.models.widgets import DateSlider, DateRangeSlider
from bokeh.plotting import figure, show

from bokeh.models.widgets import Panel, Tabs, TableColumn
from bokeh.models import CustomJS, Select
#from bokeh.themes import built_in_themes
import warnings
warnings.filterwarnings('ignore')
from operator import ge
from bokeh.io import output_notebook
from ipywidgets import interact 
import ipywidgets as widgets
from bokeh.models.axes import LogAxis, LinearAxis
output_notebook()

Loading BokehJS ...

In [199]:
css_string = f"""
.data_table {{
    display: flex;
    flex-direction: column;
    color: black;
    max-width: 80vw;
}}
"""
HTML(
    f"""
    <style>
    {css_string}
    </style>
    """
)

In [200]:
lib.datasources.fixtures(lib.datasources.fulldb()[0])

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,,Albania,41.153300,20.168300,2020-01-22,0,0,0,0,Europe
2,,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0,Africa
3,,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0,Europe
4,,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0,Africa
...,...,...,...,...,...,...,...,...,...,...
139803,,Vietnam,14.058324,108.277199,2021-06-18,12508,62,4733,7713,Western Pacific
139804,,Palestine,31.952200,35.233200,2021-06-18,312499,3548,305658,3293,Eastern Mediterranean
139805,,Yemen,15.552727,48.516388,2021-06-18,6875,1352,3846,1677,Eastern Mediterranean
139806,,Zambia,-13.133897,27.849332,2021-06-18,125157,1554,103884,19719,Africa


In [202]:
db = lib.datasources.fixtures(lib.datasources.fulldb()[0])
source = ColumnDataSource(df)

select = Select(
    name="test_select",
    title="Select A Country:",
    value="United States",
    options=list(db['Country/Region'].unique())
)

slider = DateRangeSlider(
        title='test_slider',
        name='test_slider',
        step=50,
        start=db.Date.min().date(),
        end=db.Date.max().date(),
        value=(
            db.Date.min().date() + timedelta(days=300),
            db.Date.max().date(),
        ),
        format="%Y %m %d"  
)

ctrl_pane = layout(
    children = [
        [select,],
        [slider,]
    ]
)
show(ctrl_pane)

In [203]:
def make_table():
    date_format = DateFormatter(format='RFC-1123')
    count_format = HTMLTemplateFormatter(template="""
    <div 
        style="background:<%=
        (function setBackground(){
            if(value >= 0 && value <= 2500) {
                return("green");
            }; 
            if(value >= 2500 && value <= 50000) {
                return("yellow");
            }
            if(value >=50001) {
                return("red");
            }
        }()) 
        %>;
        color: black">
        <%= value %>
    </div>
    """)


    cols = [
        TableColumn(field='Province/State', name='Province/State'),
        TableColumn(field='Country/Region', name='Country/Region'), 
        TableColumn(field='Lat', name='Lat'),
        TableColumn(field='Long', name='Long'),
        TableColumn(field='Date', name='Date', formatter=date_format),
        TableColumn(field='Confirmed', name='Confirmed', formatter=count_format),
        TableColumn(field='Deaths', name='Deaths', formatter=count_format),
        TableColumn(field='Recovered', name='Recovered'),
        TableColumn(field='Active', name='Active', formatter=count_format),
        TableColumn(field='WHO Region', name='WHO Region')
    ]
    view = CDSView(
        source=source, 
        filters=[
            BooleanFilter([
                True if (date.to_pydatetime() > datetime.fromtimestamp(slider.value[0]/1000)) & (date.to_pydatetime() < datetime.fromtimestamp(slider.value[1]/1000)) else False for date in db.Date
            ]),
            BooleanFilter([
                True if (country == select.value) & (select.value != 'all') else False for country in db['Country/Region']
            ]),
            # BooleanFilter([
            #     True if (select.value == 'all') else False for country in db['Country/Region']
            # ]),
        ]
    )
    data_table = DataTable(
        name="test",
        autosize_mode="fit_viewport",
        columns=cols,
        source=source,
        view=view,
        #index_header="Record No.",
        sizing_mode="scale_both",
        sortable=False,
        css_classes=[
            'data_table'
        ],
    )
    return data_table 



page = layout(
    children = [
        [make_table(),],
        [slider,],
    ]
)
show(page)


In [208]:
groupdb = lib.datasources.groupdb(
    db
)


daywisedb = lib.datasources.daywisedb(
    groupdb
)

#groupdb.Date

In [209]:
tmp = groupdb.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
tmp = tmp.melt(
    id_vars='Date',
    value_vars=[
        'Recovered',
        'Deaths',
        'Active'
    ],
    var_name='Case',
    value_name='Count'
)
plot = figure(
    x_range=(
        tmp.Date.min().date(),
        tmp.Date.max().date()
    )   
)
plot.grid.minor_grid_line_color = '#eeeeee'
categories = [
    'Recovered',
    'Deaths',
    'Active'
]
src = ColumnDataSource(tmp)
plot.varea_stack(
    stackers=categories,
    x='Date',
    color=brewer['Spectral'][3],
    legend_label=categories,
    source=src
)


show(plot)
tmp


,Date,Case,Count
0,2020-01-22,Recovered,30
1,2020-01-23,Recovered,32
2,2020-01-24,Recovered,39
3,2020-01-25,Recovered,42
4,2020-01-26,Recovered,56
...,...,...,...
1537,2021-06-14,Active,59589890
1538,2021-06-15,Active,59492589
1539,2021-06-16,Active,59531388
1540,2021-06-17,Active,59537875


In [210]:
tmp.Date.min().date()

datetime.date(2020, 1, 22)

In [211]:
df = pd.DataFrame(np.random.randint(10, 100, size=(15, N))).add_prefix('y')
p = figure(x_range=(0, len(df)-1), y_range=(0, 800))
p.grid.minor_grid_line_color = '#eeeeee'

names = ["y%d" % i for i in range(N)]
p.varea_stack(stackers=names, x='index', color=brewer['Spectral'][N], legend_label=names, source=df)

# reverse the legend entries to match the stacked order
p.legend.items.reverse()

show(p)

In [212]:
df.index

RangeIndex(start=0, stop=15, step=1)

In [288]:
grouped = db.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index(drop=False).set_index('Date')
grouped


,Recovered,Deaths,Active
Date,,,
2020-01-22,30,17,510
2020-01-23,32,18,605
2020-01-24,39,26,876
2020-01-25,42,42,1349
2020-01-26,56,56,2006
...,...,...,...
2021-06-14,112868176,3812083,59589890
2021-06-15,113326221,3822665,59492589
2021-06-16,113673468,3832964,59531388


In [313]:

plot = figure(
    name='test',
    title='test',
    #x_range=(grouped.index.min().date(), grouped.index.max().date()),
    #y_range=(0, grouped.Recovered.max()),
)
#plot.grid.minor_grid_line_color = '#eeeeee'

plot.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

plot.line(
    grouped.index, grouped.Active,
    color='yellow'
)
plot.line(
    grouped.index, grouped.Deaths,
    color='red'
)
plot.line(
    grouped.index, grouped.Recovered,
    color='green'
)
plot.sizing_mode = 'scale_both'
show(plot)

In [296]:
grouped.index.max().date()

datetime.date(2021, 6, 18)

In [391]:
p = figure(title='test', name='test', )
p.line(grouped.index,grouped.Deaths)
p.line(grouped.index, grouped.Recovered)
p.line(grouped.index, grouped.Active)
show(p)

In [413]:
stack = figure(title='stack', name='stack',  x_axis_type='datetime')
stack.varea(
    x=grouped.index, 
    y1=[0 for i in grouped.Deaths],
    y2=grouped.Deaths,
    fill_color='orange',
)
stack.varea(
    x=grouped.index, 
    y1=[0 for i in grouped.Recovered],
    y2=grouped.Recovered,
    fill_color='green',
)
stack.varea(
    x=grouped.index, 
    y1=[0 for i in grouped.Active],
    y2=grouped.Active,
    fill_color='yellow'
)

stack.yaxis.formatter=NumeralTickFormatter(
    num_of_ticks=len(
        grouped.Recovered.max()
    ),
    format='0.0a'
)
stack.xaxis.formatter=DatetimeTickFormatter(
        #desired_num_ticks=len(grouped.Date),
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )
show(stack)

TypeError: object of type 'numpy.int64' has no len()

In [414]:
show(stack)

In [377]:
stack

Figure(id='132438', ...)

In [390]:
stack.xaxis.ticker = DaysTicker(
    interval=len(grouped.index)/100,
    days=list(
        range(
            len(
                grouped.index
            )
        )
    )
)
#stack.y_axis_type = 'log'

AttributeError: unexpected attribute 'y_axis_type' to Figure, possible attributes are above, align, aspect_ratio, aspect_scale, background, background_fill_alpha, background_fill_color, below, border_fill_alpha, border_fill_color, center, css_classes, disabled, extra_x_ranges, extra_y_ranges, frame_height, frame_width, height, height_policy, hidpi, inner_height, inner_width, js_event_callbacks, js_property_callbacks, left, lod_factor, lod_interval, lod_threshold, lod_timeout, margin, match_aspect, max_height, max_width, min_border, min_border_bottom, min_border_left, min_border_right, min_border_top, min_height, min_width, name, outer_height, outer_width, outline_line_alpha, outline_line_cap, outline_line_color, outline_line_dash, outline_line_dash_offset, outline_line_join, outline_line_width, output_backend, renderers, reset_policy, right, sizing_mode, subscribed_events, syncable, tags, title, title_location, toolbar, toolbar_location, toolbar_sticky, visible, width, width_policy, x_

In [388]:
show(stack)